In [1]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client.companies

In [2]:
#Starting consuting time on Mongo DB

one_office = db.companies.find({    #Companies with at least 1 office
    "offices":{
        "$not":{
            "$size":0
        }
    }
},{"name":1, "offices":1})

one_office_df = pd.DataFrame(one_office)
#display(one_office_df)
print(one_office_df.shape)
one_office_df.columns


(13744, 3)


Index(['_id', 'name', 'offices'], dtype='object')

In [51]:
#Filtering with mongo
 
mongodata = db.companies.find( {"$and":[
                                {"offices.latitude": {"$exists": True}},
                                {"offices.latitude": {"$ne": None}},   
                                {"offices.longitude": {"$exists": True}},
                                {"offices.longitude": {"$ne": None}},
                                {"category_code": {"$exists": True}},
                                {"category_code": {"$ne": None}},   
                                {"founded_year": {"$exists": True}}, 
                                {"founded_year": {"$gte": 2009}},
                                {"total_money_raised": {"$exists": True}},
                                {"total_money_raised": {"$not":{"$size":0}}}, 
                                {"$or": [
                                {"category_code":  "design" } ,
                                {"category_code":  "nanotech" } , 
                                {"category_code":  "web" } , 
                                {"category_code":  "software" } , 
                                {"category_code":  "games_video" } , 
                                {"category_code":  "mobile" } , 
                                {"category_code":  "ecommerce" } ,
                                {"category_code":  "advertising" } ,
                                {"category_code":  "enterprise" } ,   
                                {"category_code":  "analytics" } ,   
                                {"category_code":  "photo_video" } ,   
                                {"category_code":  "biotech" } ]} ,
                                   
                                   
                                #{"category_code": { "tags": { "$in": ["videogame"] } }},
                                #{"category_code": { "tags": { "$in": ["design"] } }},
                                #{"category_code": { "tags": { "$in": ["nanotech"] } }}, 
                                #{"category_code": { "tags": { "$in": ["web"] } }}, 
                                #{"category_code": { "tags": { "$in": ["software"] } }}, 
                                #{"category_code": { "tags": { "$in": ["games_video"] } }}, 
                                #{"category_code": { "tags": { "$in": ["mobile"] } }}, 
                                #{"category_code": { "tags": { "$in": ["ecommerce"] } }},
                                #{"category_code": { "tags": { "$in": ["advertising"] } }},
                                #{"category_code": { "tags": { "$in": ["enterprise"] } }},   
                                #{"category_code": { "tags": { "$in": ["analytics"] } }},   
                                #{"category_code": { "tags": { "$in": ["photo_video"] } }},   
                                #{"category_code": { "tags": { "$in": ["biotech"] } }}, 
                                #{"category_code": { "tags": { "$in": ["hardware"] } }},   
                                ]
                                },
                                {
                               "_id": 0, "category_code": 1,"founded_year": 1, "name": 1, 
                               "number_of_employees": 1, "offices.country_code": 1, 
                               "offices.latitude": 1, "offices.longitude": 1, "total_money_raised": 1   
                                }
                            )
    
    
  

In [4]:
mongodata

In [5]:
mydataset = db.companies.find({"$and": 
                               [{"offices.latitude": {"$exists": True}}, 
                                {"offices.latitude": {"$ne": None}},
                                {"offices.longitude": {"$exists": True}}, 
                                {"offices.longitude": {"$ne": None}},
                                {"category_code": {"$exists": True}}, 
                                {"category_code": {"$ne": None}},
                                {"number_of_employees": {"$exists": True}}, 
                                {"number_of_employees": {"$ne": None}},
                                {"founded_year": {"$exists": True}}, 
                                {"founded_year": {"$gte": 2008}}]}, 
                              {"_id": 0, "category_code": 1,"founded_year": 1, "name": 1, 
                               "number_of_employees": 1, "offices.country_code": 1, 
                               "offices.latitude": 1, "offices.longitude": 1, "total_money_raised": 1})

In [52]:
df = pd.DataFrame(mongodata)
df

,category_code,founded_year,name,number_of_employees,offices,total_money_raised
0,web,2011,Mokitown,NaN,"[{'country_code': 'USA', 'latitude': 37.09024,...",$0
1,mobile,2010,GENWI,25.0,"[{'country_code': 'USA', 'latitude': 33.8171, ...",$7.1M
2,web,2013,Fixya,30.0,"[{'country_code': 'USA', 'latitude': 37.566879...",$8M
3,games_video,2009,alluc,7.0,"[{'country_code': 'DEU', 'latitude': 53.707739...",$0
4,enterprise,2010,Needium,15.0,"[{'country_code': 'CAN', 'latitude': 45.504930...",$1M
5,ecommerce,2011,RazorGator,NaN,"[{'country_code': 'USA', 'latitude': 34.047312...",$58.8M
6,advertising,2013,Advaliant,60.0,"[{'country_code': 'CAN', 'latitude': 43.646599...",$100k
7,games_video,2011,Social Gaming Network,100.0,"[{'country_code': 'USA', 'latitude': 37.446823...",$17.1M
8,analytics,2010,YouNoodle,18.0,"[{'country_code': 'USA', 'latitude': 37.776658...",$0
9,games_video,2009,Crootpad,2.0,"[{'country_code': 'USA', 'latitude': 42.091078...",$0


In [7]:
def getFirstOffice(data):
    data = data['offices']
    #Only create the geoJSON object if all geodata is available
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }
    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


first_office = one_office_df[["offices"]].apply(getFirstOffice, result_type = "expand", axis = 1)
print(first_office.shape)
display(first_office.head())

(13744, 4)


,lat,lng,oficina_principal,totalOffices
0,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
1,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",2.0
2,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
3,NaN,NaN,NaN,1.0
4,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1.0


In [8]:
df_clean = pd.concat([one_office_df,first_office], axis=1)[["name","lat","lng", "oficina_principal","totalOffices"]]

display(df_clean.head())


,name,lat,lng,oficina_principal,totalOffices
0,AdventNet,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
1,Wetpaint,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,...",2.0
2,Zoho,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,...",1.0
3,Omnidrive,NaN,NaN,NaN,1.0
4,Geni,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1.0


In [9]:
def drop_null(df):
    return df.dropna(axis=0)

In [10]:
df_cleaned = df_clean.dropna()

In [11]:
df_cleaned.head()
df_cleaned.shape


(9618, 5)